Ex.1
Un joc intre doi jucatori, J0 si J2, se desfășoară in felul următor:
-se arunca mai întâi cu o moneda (normală) pentru a decide cine începe J0 sau J1
-in prima runda,jucatorul desemnat arunca cu propria moneda; fie n aparține {0, 1} numărul de steme obținute
-in a doua runda, celălalt jucător arunca o moneda proprie n+1 ori; fie m numărul de steme obținute.
Jucătorul din prima runda câștigă daca n>=m, in caz contrar castigand jucătorul din a doua runda.
1. Estimați care dintre dei doi jucatori are șansele cele mai mari de câștig, simulând un joc de 1000 ori.
2. Folosind pgmpy, definiți o rețea Bayesiana care sa descrie contextul de mai sus.
3. Folosind modelul de mai sus, determinați cine e cel mai probabil sa fi început jocul, știind ca in a doua runda s-a obținut o singura stema.

In [10]:
import random

from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination


# ---------------------------------------------------------
# 1. SIMULARE JOC (1000 de runde)
# ---------------------------------------------------------

def joaca_un_joc(rng):
    """
    Simuleaza UN joc complet.
    Returneaza:
      - starter: cine a inceput ('J0' sau 'J1')
      - winner:  cine a castigat ('J0' sau 'J1')
      - role_winner: 'first' sau 'second' (rolul jucatorului castigator)
    """
    # alegem aleator cine incepe: J0 sau J1
    starter = "J0" if rng.random() < 0.5 else "J1"
    other = "J1" if starter == "J0" else "J0"

    # runda 1: jucatorul din prima runda arunca o data (moneda normala)
    n = 1 if rng.random() < 0.5 else 0  # nr. de steme (0 sau 1)

    # runda a 2-a: celalalt arunca de n+1 ori (tot cu moneda normala)
    m = 0
    for _ in range(n + 1):
        if rng.random() < 0.5:
            m += 1

    # castiga jucatorul din prima runda daca n >= m, altfel cel din a doua
    if n >= m:
        winner = starter
        role_winner = "first"
    else:
        winner = other
        role_winner = "second"

    return starter, winner, role_winner


def simuleaza_jocuri(num_games=1000, seed=0):
    rng = random.Random(seed)
    wins = {"J0": 0, "J1": 0}
    wins_role = {"first": 0, "second": 0}

    for _ in range(num_games):
        starter, winner, role = joaca_un_joc(rng)
        wins[winner] += 1
        wins_role[role] += 1

    print(f"--- SIMULARE pentru {num_games} jocuri ---")
    print("Victorii J0:", wins["J0"])
    print("Victorii J1:", wins["J1"])
    print("Jucatorul din PRIMA runda a castigat:", wins_role["first"])
    print("Jucatorul din A DOUA runda a castigat:", wins_role["second"])
    print("Procent (prima runda):", wins_role["first"] / num_games)
    print("Procent (a doua runda):", wins_role["second"] / num_games)
    print()


# ---------------------------------------------------------
# 2. RETEA BAYESIANA cu pgmpy
# ---------------------------------------------------------

def construieste_model_bayesian():
    """
    Construieste modelul Bayes-ian pentru joc:
      S - cine incepe (0=J0, 1=J1)
      N - nr. de steme in prima runda (0/1)
      M - nr. de steme in a doua runda (0/1/2)
      W - castigatorul (0=J0, 1=J1)
    """

    model = BayesianModel()

    # in versiunile vechi nu exista add_edges_from, folosim add_edge
    edges = [
        ('S', 'N'),
        ('N', 'M'),
        ('S', 'W'),
        ('N', 'W'),
        ('M', 'W')
    ]
    for u, v in edges:
        model.add_edge(u, v)

    # 2.1. CPD pentru S (cine incepe): moneda normala
    # S = 0 -> J0, S = 1 -> J1
    cpd_S = TabularCPD(
        variable='S',
        variable_card=2,
        values=[[0.5],
                [0.5]]
    )

    # 2.2. CPD pentru N (nr. steme prima runda)
    # N este independent de S, dar il scriem conditionat cu distributie egala
    # coloana 0: S=0, coloana 1: S=1
    # rand 0: P(N=0|S), rand 1: P(N=1|S)
    cpd_N = TabularCPD(
        variable='N',
        variable_card=2,
        values=[
            [0.5, 0.5],  # N=0
            [0.5, 0.5]   # N=1
        ],
        evidence=['S'],
        evidence_card=[2]
    )

    # 2.3. CPD pentru M (nr. steme runda a doua) – depinde de N
    # M are valori {0,1,2}
    # Daca N=0 -> 1 aruncare: M=0 cu 0.5, M=1 cu 0.5, M=2 cu 0
    # Daca N=1 -> 2 aruncari: M ~ Bin(2, 0.5) => [0.25, 0.5, 0.25]
    # coloana 0: N=0, coloana 1: N=1
    cpd_M = TabularCPD(
        variable='M',
        variable_card=3,
        values=[
            [0.5, 0.25],  # M=0
            [0.5, 0.5],   # M=1
            [0.0, 0.25]   # M=2
        ],
        evidence=['N'],
        evidence_card=[2]
    )

    # 2.4. CPD pentru W (castigator) – determinist in functie de S, N, M:
    # W = 0 -> J0, W = 1 -> J1
    # castiga starter-ul daca N >= M, altfel celalalt
    values_W_0 = []  # P(W=0|S,N,M)
    values_W_1 = []  # P(W=1|S,N,M)

    # S in {0,1}, N in {0,1}, M in {0,1,2}
    for s in [0, 1]:
        for n in [0, 1]:
            for m in [0, 1, 2]:
                starter = s          # 0=J0, 1=J1
                other = 1 - s
                winner = starter if n >= m else other

                # W=0 ?
                values_W_0.append(1.0 if winner == 0 else 0.0)
                # W=1 ?
                values_W_1.append(1.0 if winner == 1 else 0.0)

    cpd_W = TabularCPD(
        variable='W',
        variable_card=2,
        values=[values_W_0, values_W_1],
        evidence=['S', 'N', 'M'],
        evidence_card=[2, 2, 3]
    )

    # 2.5. Adaugam CPD-urile in model
    model.add_cpds(cpd_S, cpd_N, cpd_M, cpd_W)

    # verificam ca modelul e corect
    assert model.check_model(), "Modelul Bayes-ian nu este valid!"

    return model


def interogare_p_s_dat_m1(model):
    """
    Foloseste inferenta in reteaua Bayesiana pentru a calcula P(S | M=1),
    adica: cine e mai probabil sa fi inceput jocul, stiind ca in a doua runda
    s-a obtinut o singura stema.
    """
    infer = VariableElimination(model)

    # in versiunile vechi: query(['S'], evidence={'M':1}) returneaza un dict
    rezultat = infer.query(['S'], evidence={'M': 1})
    phi_S = rezultat['S']  # factorul pentru S

    # phi_S.values este un array de lungime 2: [P(S=0|M=1), P(S=1|M=1)]
    p_S0 = phi_S.values[0]  # S=0 => J0
    p_S1 = phi_S.values[1]  # S=1 => J1

    print("--- POSTERIOR pentru S, stiind M=1 ---")
    print(f"P(S = J0 | M = 1) = {p_S0}")
    print(f"P(S = J1 | M = 1) = {p_S1}")
    print()


# ---------------------------------------------------------
# MAIN
# ---------------------------------------------------------

if __name__ == "__main__":
    # 1. Simulare
    simuleaza_jocuri(num_games=1000, seed=0)

    # 2. Retea Bayesiana
    model = construieste_model_bayesian()

    # 3. Interogare: cine e mai probabil sa fi inceput, stiind ca M=1
    interogare_p_s_dat_m1(model)


--- SIMULARE pentru 1000 jocuri ---
Victorii J0: 512
Victorii J1: 488
Jucatorul din PRIMA runda a castigat: 612
Jucatorul din A DOUA runda a castigat: 388
Procent (prima runda): 0.612
Procent (a doua runda): 0.388



AttributeError: 'BayesianModel' object has no attribute 'add_edge'

Ex.2
Sa presupunem ca administrezi o banca. Un aspect importantpentru satisfactia clientilor este timpul mediu de asteptare la coada, pe care vrei sa il estimezi.
#Pentru aceasta propunem un model de inferenta Bayesiana in felul urmator:
#-timpul mediu de asteptare este modelat de o distributie normala de parametri α si μ.
#1. Generati 100 de timpi medii de astepatare folosind distributia a priori alese de voi.
#2. Descrieti modelul in PyMC, folosind ca distributii a priori alese de voi. Cum justificati alegerea facuta?
#3. Estimati cu ajutorul modelului de mai sus, distributia a posteriori pentru parametrul μ. Corespunde acesta asteptarilor voastre? A se folosi o vizualizare grafica

In [ ]:
import numpy as np
import pymc as pm        # daca ai PyMC3, foloseste: import pymc3 as pm
import matplotlib.pyplot as plt


# 1. Generare de date din prior (100 de timpi medii)

# setam un seed pt reproductibilitate
rng = np.random.default_rng(0)

# alegere prior pentru μ si α (precizie)
mu_prior_mean = 10.0    # credem ca media e cam 10 minute
mu_prior_sigma = 5.0    # incertitudine mare (dispersie larga)

alpha_shape = 2.0       # parametru "alpha" (forma) pentru Gamma
alpha_rate = 1.0        # parametru "beta" (rata) pentru Gamma

# extragem "valori adevarate" din prior (asta e, de fapt, o simulare din prior)
mu_true = rng.normal(mu_prior_mean, mu_prior_sigma)
alpha_true = rng.gamma(shape=alpha_shape, scale=1.0/alpha_rate)  # gamma(shape, scale)
sigma_true = 1.0 / np.sqrt(alpha_true)

print(f"Valoare simulata pentru μ (din prior): {mu_true:.3f}")
print(f"Valoare simulata pentru α (din prior): {alpha_true:.3f}")
print(f" => sigma_true = {sigma_true:.3f}")

# generam 100 de timpi medii de asteptare din distributia Normala(μ, σ^2)
n_obs = 100
waiting_times = rng.normal(loc=mu_true, scale=sigma_true, size=n_obs)

print(f"\nAm generat {n_obs} timpi medii de asteptare.")
print(f"Media empirica a datelor simulate: {waiting_times.mean():.3f}")
print(f"Deviatia standard empirica: {waiting_times.std(ddof=1):.3f}\n")


# ---------------------------------------------------------
# 2. Descrierea modelului in PyMC (prior + verosimilitate)
# ---------------------------------------------------------
# Model:
#   μ ~ Normal(mu_prior_mean, mu_prior_sigma^2)
#   α ~ Gamma(alpha_shape, alpha_rate)
#   σ = 1 / sqrt(α)
#   waiting_times ~ Normal(μ, σ^2)

with pm.Model() as model:
    # prior pentru μ: normal, destul de "larg"
    mu = pm.Normal("mu", mu=mu_prior_mean, sigma=mu_prior_sigma)

    # prior pentru α: Gamma(forma, rata), pozitiv -> potrivit pentru precizie
    alpha = pm.Gamma("alpha", alpha=alpha_shape, beta=alpha_rate)

    # definim σ din α (σ = 1 / sqrt(α))
    sigma = pm.Deterministic("sigma", 1.0 / pm.math.sqrt(alpha))

    # verosimilitatea datelor: timpii observati sunt Normali(μ, σ)
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=waiting_times)

    # ---------------------------------------------------------
    # 3. Estimarea distributiei a posteriori pentru μ
    # ---------------------------------------------------------
    trace = pm.sample(
        2000,              # numar de esantioane MCMC
        tune=1000,         # numar de iteratii de incalzire (burn-in)
        target_accept=0.9, # putin mai mare sa avem lanturi mai stabile
        random_seed=0,
        return_inferencedata=False  # ca sa primim un dict simplu
    )

# extragem valorile mostenate din posterior pentru μ
posterior_mu = trace["mu"]

print(f"Posterior μ: medie ≈ {posterior_mu.mean():.3f}, std ≈ {posterior_mu.std(ddof=1):.3f}")


# ---------------------------------------------------------
# Vizualizare grafica a distributiei a posteriori pentru μ
# ---------------------------------------------------------

plt.figure()
plt.hist(posterior_mu, bins=30, density=True)
plt.axvline(mu_true, linestyle="--", label="μ (valoare simulata din prior)")
plt.title("Distributia a posteriori pentru μ")
plt.xlabel("μ")
plt.ylabel("Densitate")
plt.legend()
plt.tight_layout()
plt.show()


INFO:arviz.preview:arviz_base not installed
INFO:arviz.preview:arviz_stats not installed
INFO:arviz.preview:arviz_plots not installed


Valoare simulata pentru μ (din prior): 10.629
Valoare simulata pentru α (din prior): 1.502
 => sigma_true = 0.816

Am generat 100 timpi medii de asteptare.
Media empirica a datelor simulate: 10.700
Deviatia standard empirica: 0.792



INFO:pymc.sampling.mcmc:Initializing NUTS using jitter+adapt_diag...
INFO:pymc.sampling.mcmc:Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc.sampling.mcmc:NUTS: [mu, alpha]


Output()